In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_data = pd.read_pickle('./data/train_prepared_1')
test_data = pd.read_pickle('./data/test_prepared_1')

In [4]:
train_data.head()

,ID,age,failures,absences,G1,G2,G3,score_average,traveltime,studytime,...,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other,alc_average
0,136,-0.585112,-0.378483,-0.070919,-0.750328,-0.702440,-0.107537,-0.514094,1.000000,0.000000,...,0,0,1,0,0,0,0,1,0,1.000000
1,256,0.234634,-0.378483,0.244909,0.923974,0.212119,0.413003,0.522787,0.000000,1.000000,...,1,0,0,0,0,1,0,1,0,1.000000
2,350,1.874125,4.315080,0.560737,-1.085188,-1.312146,-0.888347,-1.136223,0.666667,0.333333,...,1,0,0,1,0,0,0,0,1,3.000000
3,60,-0.585112,-0.378483,0.244909,-0.415467,-0.092734,-0.107537,-0.203030,0.000000,0.333333,...,0,1,0,0,1,0,0,1,0,2.285714
4,535,-1.404857,-0.378483,1.034479,-0.750328,-0.092734,-0.367807,-0.410406,0.666667,0.333333,...,1,0,1,0,0,0,0,1,0,1.571429


In [5]:
test_data.head()

,ID,age,failures,absences,G1,G2,G3,score_average,traveltime,studytime,...,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other,alc_average
0,576,1.054379,-0.378483,0.560737,-0.750328,-0.397587,-0.107537,-0.410406,0.333333,0.000000,...,0,0,1,0,0,0,0,1,0,5.000000
1,280,0.234634,-0.378483,4.034846,-1.085188,-1.007293,-0.888347,-1.032535,0.333333,0.000000,...,0,0,0,1,0,0,0,1,0,2.571429
2,141,-0.585112,-0.378483,-0.702576,0.923974,0.516972,0.413003,0.626475,1.000000,0.333333,...,0,0,1,0,0,0,0,1,0,1.000000
3,429,-0.585112,-0.378483,0.086995,-0.415467,-0.092734,-0.107537,-0.203030,0.000000,0.333333,...,0,0,0,1,0,0,0,1,0,1.571429
4,249,-0.585112,-0.378483,-0.702576,0.254253,0.212119,0.152733,0.211722,0.333333,0.666667,...,0,0,0,0,0,1,0,1,0,1.571429


# Base Model selection

In [37]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor

from sklearn.naive_bayes import BernoulliNB #향후 분류 용도
from sklearn.naive_bayes import GaussianNB #향후 분류 용도

from sklearn import svm
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [38]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

In [50]:
from sklearn.metrics import make_scorer
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [46]:
def rmse(y_true, y_pred):
    return np.sqrt(((y_pred - y_true)**2).mean())

In [64]:
 train_data.iloc[:,1:-1].values

array([[-0.58511174, -0.37848294, -0.07091948, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.23463373, -0.37848294,  0.24490861, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.87412469,  4.31508023,  0.56073671, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-1.40485722, -0.37848294, -0.70257567, ...,  0.        ,
         1.        ,  0.        ],
       [-1.40485722, -0.37848294,  0.24490861, ...,  1.        ,
         0.        ,  0.        ],
       [ 1.05437921,  1.18603812,  0.71865076, ...,  1.        ,
         0.        ,  0.        ]])

In [79]:
kfold = KFold(n_splits=10 , random_state=22)
r2_average_cv = []
r2 = []
r2_std_cv=[]

nmse_average_cv=[]
nmse=[]
nmse_std_cv=[]

classifiers = ['LinearRegression','Ridge','Lasso','ElasticNet', 'SGDRegressor', 
                'SVM_LSVR','SVM_NuSVR', 'SVM_SVR',
                'DecisionTreeRegressor', 'RandomForestRegressor']
models = [LinearRegression(),Ridge(),Lasso(),ElasticNet(),SGDRegressor(), 
           svm.LinearSVR(), svm.NuSVR(),svm.SVR(), 
           DecisionTreeRegressor(),RandomForestRegressor(n_estimators=100)]

scoring=['r2', 'neg_median_absolute_error']

for model in models:
    cv_result = cross_validate(estimator=model, X=train_data.iloc[:,1:-1].values, y = train_data.iloc[:,-1].values, 
                               cv = kfold, scoring=scoring,verbose=1)
##     cv_result = cross_val_score(model,train_data.iloc[:,1:-1], train_data.iloc[:,-1], cv = kfold, scoring=['r2','neg_mean_squared_error'])
    r2_average_cv.append(cv_result['test_r2'].mean())
    r2_std_cv.append(cv_result['test_r2'].std())
    r2.append(cv_result['test_r2'])
    
    nmse_average_cv.append(cv_result['test_neg_median_absolute_error'].mean())
    nmse_std_cv.append(cv_result['test_neg_median_absolute_error'].std())
    nmse.append(cv_result['test_neg_median_absolute_error'])
    
baseline_estimate_df = pd.DataFrame({'r2_average_cv' : r2_average_cv,
#                                      'r2':r2,
                                     'r2_std_cv':r2_std_cv,
                                     'nmse_average_cv':nmse_average_cv,
#                                      'nmse':nmse,
                                     'nmse_std_cv':nmse_std_cv}, index = classifiers)
baseline_estimate_df

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'skl

,nmse_average_cv,nmse_std_cv,r2_average_cv,r2_std_cv
LinearRegression,-1.393330e-15,5.898043e-16,1.000000,0.000000e+00
Ridge,-9.890869e-03,2.332046e-03,0.999484,1.233708e-04
Lasso,-6.787335e-01,8.112551e-02,-0.014890,2.150341e-02
ElasticNet,-6.787335e-01,8.112551e-02,-0.014890,2.150341e-02
SGDRegressor,-2.796637e-01,4.658222e-02,0.730144,3.077791e-02
SVM_LSVR,-2.081389e-05,5.680051e-06,1.000000,4.887754e-10
SVM_NuSVR,-8.371362e-02,1.313807e-02,0.929171,2.348151e-02
SVM_SVR,-1.073403e-01,1.424407e-02,0.916003,2.358350e-02
DecisionTreeRegressor,-6.661338e-17,1.421779e-16,0.998454,2.063094e-03
RandomForestRegressor,-1.243450e-15,5.457060e-16,0.997997,3.200521e-03


* 전반적으로 설명력이 좋은 모델로 보여진다.
* 그렇다면 이번에는 랜덤 포레스트로 어떤 칼럼이 설명력을 크게 가지는지 보도록 하겠다.